In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("data/stream_e.csv", sep = "ef5r64i", engine = "python")
print(df.columns, df.shape)


In [ ]:
#Creating the dict with unique users and a list for the tweets they have written
corpus = {}
for i in range(df.shape[0]):
    if df.iloc[i]["NombreUsuario"] in corpus:
        corpus[df.iloc[i]["NombreUsuario"]].append(df.iloc[i]["Texto"])
    else:
        corpus[df.iloc[i]["NombreUsuario"]] = [df.iloc[i]["Texto"]]
type(corpus), len(corpus)

In [ ]:
documents = list(df.Texto)

In [ ]:
#LSA

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
  
# raw documents to tf-idf matrix: 
vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)
# SVD to reduce dimensionality: 
svd_model = TruncatedSVD(n_components=100,         # num dimensions
                         algorithm='randomized',
                         n_iter=10)
# pipeline of tf-idf + SVD, fit to and applied to documents:
svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])
svd_matrix = svd_transformer.fit_transform(documents)

# svd_matrix can later be used to compare documents, compare words, or compare queries with documents

In [ ]:
#LDA

import gensim

from gensim.corpora.Dictionary import load_from_text, doc2bow
from gensim.corpora import MmCorpus
from gensim.models.ldamodel import LdaModel
document = "This is some document..."
# load id->word mapping (the dictionary)
id2word = load_from_text('wiki_en_wordids.txt')
# load corpus iterator
mm = MmCorpus('wiki_en_tfidf.mm')
# extract 100 LDA topics, updating once every 10,000
lda = LdaModel(corpus=mm, id2word=id2word, num_topics=100, update_every=1, chunksize=10000, passes=1)
# use LDA model: transform new doc to bag-of-words, then apply lda
doc_bow = doc2bow(document.split())
doc_lda = lda[doc_bow]
# doc_lda is vector of length num_topics representing weighted presence of each topic in the doc

---------

## generando un txt por usuario con los tweets

In [1]:
import pandas as pd
import pyodbc 
import configLocal

server = configLocal.server
database = configLocal.database
username = configLocal.username
password = configLocal.password
cnxn = configLocal.cnxn
cursor = cnxn.cursor()

In [2]:
data = pd.read_sql_query("select msg_id, msg_users_id, msg_timestamp, msg_text from dbo.msg", cnxn)

In [3]:
data.head()

,msg_id,msg_users_id,msg_timestamp,msg_text
0,1123283650638942211,428308122,1556646666570,@aritzeta @bibiloni @JosepMClD @Ratafia_Party ...
1,1123283654740983808,1669818008,1556646667548,Que fuerte!!! Y que lastima
2,1123283664123703296,2295120738,1556646669785,"@Helga_tito Bro no me quejo, si me parece de p..."
3,1123283666682220544,3327214991,1556646670395,@iescolar Evidente!!!
4,1123283688136032256,141861159,1556646675510,Karen me manda msj todos los días ??


In [31]:
corpus = {}
for i in range(data.shape[0]):
    if data.iloc[i]["msg_users_id"] in corpus:
        corpus[data.iloc[i]["msg_users_id"]].append(data.iloc[i]["msg_text"])
    else:
        corpus[data.iloc[i]["msg_users_id"]] = [data.iloc[i]["msg_text"]]
type(corpus), len(corpus)

(dict, 4184)

In [14]:
type(corpus["428308122"])

list

In [24]:
a = corpus["428308122"]

In [47]:
for i in corpus:
    with open("C:/Users/alonsoga/OneDrive - HP Inc/Oficina/Documentacion/DS/RoundU/data/user_twit/{}.txt".format(i), "x") as file:
        file.write(" ".join(corpus[i]))

---------

In [ ]:
corpus = []

for i in range(len(data)):
    corpus.append(data.iloc[i]["msg_text"])

In [ ]:
import gensim
import logging
import tempfile
import os

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

from gensim import corpora
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
from nltk.corpus import stopwords
from string import punctuation

# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('spanish') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

In [ ]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'elon.dict'))  # store the dictionary, for future reference
print(dictionary)

In [ ]:
print(dictionary.token2id)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)  # store to disk, for later use

In [ ]:
from gensim import corpora, models, similarities


In [ ]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model


In [ ]:
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors


LDA

In [ ]:
total_topics = 10


In [ ]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

In [ ]:
#Show first n important word in the topics:
lda.show_topics(total_topics,10)

In [ ]:
from collections import OrderedDict

data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}
#data_lda

In [ ]:
import pandas as pd

df_lda = pd.DataFrame(data_lda)
print(df_lda.shape)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

In [ ]:
df_lda

In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

---------

extract entities: https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [ ]:
ex = 'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices'

In [ ]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent, language="spanish")
    sent = nltk.pos_tag(sent, lang="spanish")
    return sent

In [ ]:
sent = preprocess(ex)
sent

In [ ]:
ex = str([word for word in corpus])

In [ ]:
preprocess(corpus[0])

https://stackoverflow.com/questions/14732465/nltk-tagging-spanish-words-using-a-corpus  
    https://www.quora.com/Which-languages-are-available-in-nltk-part-of-speech-tagger

-----------


In [2]:
import nltk

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [6]:
from nltk.corpus import gutenberg

In [8]:
print (gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [10]:
from nltk.corpus import gutenberg
for fileid in gutenberg.fileids():
    num_chars = len(gutenberg.raw(fileid))
    num_tokens = len(gutenberg.words(fileid))
    num_sents = len(gutenberg.sents(fileid))
    num_vocab = len(set(w.lower() for w in gutenberg.words(fileid)))
    print (str(num_chars) +  "-" + str(num_tokens) + "-" + str(num_sents) +  "-" + str(num_vocab))

887071-192427-7752-7344
466292-98171-3747-5835
673022-141576-4999-6403
4332554-1010654-30103-12767
38153-8354-438-1535
249439-55563-2863-3940
84663-18963-1054-1559
144395-34110-1703-2636
457450-96996-4779-8335
406629-86063-3806-7794
320525-69213-3742-6349
935158-210663-10230-8447
1242990-260819-10059-17231
468220-96825-1851-9021
112310-25833-2163-3032
162881-37360-3106-4716
100351-23140-1907-3464
711215-154883-4250-12452


In [1]:
from nltk.corpus import PlaintextCorpusReader
import nltk
wordlists = PlaintextCorpusReader("C:/Users/alonsoga/OneDrive - HP Inc/Oficina/Documentacion/DS/RoundU/data/user_twit" ,".*\.txt", 
                                  encoding="Latin1")
#wordlists.words('stream_e.csv')
print ("Sentences:" +str(len(wordlists.sents())))
for sentence in wordlists.sents():
    tokens = nltk.word_tokenize(str(sentence))
    tagged_tokens = nltk.pos_tag(tokens)
    verbs = 0
    for tagged in tagged_tokens:
        token, tag = tagged
        if tag == 'VBP' or tag == 'VB':
            verbs += 1
    print ("Tokens:" +str(len(tokens)) +  "-" +"Verbs:"+str(verbs))

Sentences:24112
Tokens:125-Verbs:0
Tokens:20-Verbs:0
Tokens:25-Verbs:0
Tokens:186-Verbs:0
Tokens:323-Verbs:0
Tokens:57-Verbs:0
Tokens:20-Verbs:0
Tokens:55-Verbs:0
Tokens:106-Verbs:0
Tokens:20-Verbs:0
Tokens:55-Verbs:0
Tokens:147-Verbs:0
Tokens:38-Verbs:0
Tokens:28-Verbs:0
Tokens:32-Verbs:0
Tokens:24-Verbs:0
Tokens:11-Verbs:0
Tokens:5-Verbs:0
Tokens:26-Verbs:0
Tokens:19-Verbs:0
Tokens:49-Verbs:0
Tokens:45-Verbs:0
Tokens:17-Verbs:0
Tokens:28-Verbs:0
Tokens:48-Verbs:0
Tokens:20-Verbs:0
Tokens:39-Verbs:0
Tokens:29-Verbs:0
Tokens:18-Verbs:0
Tokens:76-Verbs:0
Tokens:92-Verbs:0
Tokens:84-Verbs:0
Tokens:20-Verbs:0
Tokens:25-Verbs:0
Tokens:118-Verbs:0
Tokens:44-Verbs:0
Tokens:29-Verbs:0
Tokens:135-Verbs:0
Tokens:26-Verbs:0
Tokens:41-Verbs:0
Tokens:33-Verbs:0
Tokens:28-Verbs:0
Tokens:91-Verbs:0
Tokens:40-Verbs:0
Tokens:42-Verbs:0
Tokens:119-Verbs:0
Tokens:20-Verbs:0
Tokens:51-Verbs:0
Tokens:37-Verbs:0
Tokens:43-Verbs:0
Tokens:57-Verbs:0
Tokens:24-Verbs:0
Tokens:122-Verbs:0
Tokens:36-Verbs:0
Toke

Tokens:96-Verbs:0
Tokens:22-Verbs:0
Tokens:39-Verbs:0
Tokens:24-Verbs:0
Tokens:73-Verbs:0
Tokens:52-Verbs:0
Tokens:45-Verbs:0
Tokens:42-Verbs:0
Tokens:20-Verbs:0
Tokens:41-Verbs:0
Tokens:26-Verbs:1
Tokens:79-Verbs:0
Tokens:69-Verbs:0
Tokens:71-Verbs:0
Tokens:35-Verbs:0
Tokens:26-Verbs:0
Tokens:122-Verbs:0
Tokens:110-Verbs:0
Tokens:120-Verbs:0
Tokens:33-Verbs:0
Tokens:78-Verbs:0
Tokens:37-Verbs:0
Tokens:6-Verbs:0
Tokens:11-Verbs:0
Tokens:47-Verbs:0
Tokens:29-Verbs:0
Tokens:42-Verbs:0
Tokens:49-Verbs:0
Tokens:6-Verbs:0
Tokens:28-Verbs:0
Tokens:54-Verbs:0
Tokens:17-Verbs:0
Tokens:52-Verbs:0
Tokens:45-Verbs:0
Tokens:28-Verbs:0
Tokens:17-Verbs:0
Tokens:76-Verbs:0
Tokens:50-Verbs:0
Tokens:38-Verbs:0
Tokens:14-Verbs:0
Tokens:93-Verbs:0
Tokens:6-Verbs:0
Tokens:29-Verbs:0
Tokens:6-Verbs:0
Tokens:45-Verbs:0
Tokens:59-Verbs:0
Tokens:278-Verbs:0
Tokens:71-Verbs:0
Tokens:76-Verbs:0
Tokens:26-Verbs:0
Tokens:32-Verbs:0
Tokens:229-Verbs:0
Tokens:106-Verbs:0
Tokens:99-Verbs:0
Tokens:20-Verbs:0
Tokens:1

Tokens:46-Verbs:0
Tokens:47-Verbs:0
Tokens:18-Verbs:0
Tokens:33-Verbs:0
Tokens:48-Verbs:0
Tokens:40-Verbs:0
Tokens:45-Verbs:0
Tokens:55-Verbs:0
Tokens:18-Verbs:0
Tokens:53-Verbs:0
Tokens:61-Verbs:0
Tokens:73-Verbs:0
Tokens:46-Verbs:0
Tokens:58-Verbs:0
Tokens:25-Verbs:0
Tokens:70-Verbs:0
Tokens:23-Verbs:0
Tokens:32-Verbs:0
Tokens:27-Verbs:0
Tokens:29-Verbs:0
Tokens:39-Verbs:0
Tokens:34-Verbs:0
Tokens:43-Verbs:0
Tokens:53-Verbs:0
Tokens:71-Verbs:0
Tokens:96-Verbs:0
Tokens:21-Verbs:0
Tokens:64-Verbs:0
Tokens:114-Verbs:0
Tokens:64-Verbs:0
Tokens:154-Verbs:0
Tokens:77-Verbs:0
Tokens:17-Verbs:0
Tokens:74-Verbs:0
Tokens:71-Verbs:0
Tokens:125-Verbs:0
Tokens:61-Verbs:0
Tokens:71-Verbs:0
Tokens:95-Verbs:0
Tokens:46-Verbs:0
Tokens:43-Verbs:0
Tokens:53-Verbs:0
Tokens:58-Verbs:0
Tokens:43-Verbs:0
Tokens:43-Verbs:0
Tokens:43-Verbs:0
Tokens:43-Verbs:0
Tokens:94-Verbs:0
Tokens:85-Verbs:0
Tokens:57-Verbs:0
Tokens:60-Verbs:0
Tokens:30-Verbs:0
Tokens:5-Verbs:0
Tokens:201-Verbs:0
Tokens:36-Verbs:0
Tokens:

Tokens:53-Verbs:0
Tokens:114-Verbs:0
Tokens:302-Verbs:0
Tokens:96-Verbs:0
Tokens:34-Verbs:0
Tokens:40-Verbs:0
Tokens:36-Verbs:0
Tokens:85-Verbs:0
Tokens:46-Verbs:0
Tokens:26-Verbs:0
Tokens:42-Verbs:0
Tokens:66-Verbs:0
Tokens:59-Verbs:0
Tokens:64-Verbs:0
Tokens:77-Verbs:0
Tokens:30-Verbs:0
Tokens:38-Verbs:0
Tokens:118-Verbs:0
Tokens:68-Verbs:0
Tokens:35-Verbs:0
Tokens:63-Verbs:0
Tokens:75-Verbs:0
Tokens:88-Verbs:0
Tokens:73-Verbs:0
Tokens:88-Verbs:0
Tokens:88-Verbs:0
Tokens:40-Verbs:0
Tokens:33-Verbs:0
Tokens:40-Verbs:0
Tokens:11-Verbs:0
Tokens:53-Verbs:0
Tokens:53-Verbs:0
Tokens:53-Verbs:0
Tokens:23-Verbs:0
Tokens:59-Verbs:0
Tokens:103-Verbs:0
Tokens:46-Verbs:0
Tokens:156-Verbs:1
Tokens:40-Verbs:0
Tokens:56-Verbs:0
Tokens:28-Verbs:0
Tokens:83-Verbs:0
Tokens:134-Verbs:0
Tokens:53-Verbs:0
Tokens:85-Verbs:0
Tokens:59-Verbs:0
Tokens:75-Verbs:0
Tokens:44-Verbs:0
Tokens:87-Verbs:0
Tokens:126-Verbs:0
Tokens:32-Verbs:0
Tokens:31-Verbs:0
Tokens:25-Verbs:0
Tokens:75-Verbs:0
Tokens:42-Verbs:0
Tok

Tokens:28-Verbs:0
Tokens:56-Verbs:0
Tokens:20-Verbs:0
Tokens:30-Verbs:0
Tokens:27-Verbs:0
Tokens:84-Verbs:0
Tokens:27-Verbs:0
Tokens:118-Verbs:0
Tokens:46-Verbs:0
Tokens:14-Verbs:0
Tokens:107-Verbs:0
Tokens:42-Verbs:0
Tokens:148-Verbs:0
Tokens:46-Verbs:0
Tokens:26-Verbs:0
Tokens:36-Verbs:0
Tokens:26-Verbs:0
Tokens:17-Verbs:0
Tokens:15-Verbs:0
Tokens:45-Verbs:0
Tokens:27-Verbs:0
Tokens:43-Verbs:1
Tokens:135-Verbs:0
Tokens:14-Verbs:0
Tokens:49-Verbs:0
Tokens:14-Verbs:0
Tokens:14-Verbs:0
Tokens:44-Verbs:0
Tokens:95-Verbs:0
Tokens:11-Verbs:0
Tokens:41-Verbs:0
Tokens:40-Verbs:0
Tokens:40-Verbs:0
Tokens:56-Verbs:0
Tokens:31-Verbs:0
Tokens:8-Verbs:0
Tokens:85-Verbs:0
Tokens:49-Verbs:0
Tokens:10-Verbs:0
Tokens:58-Verbs:0
Tokens:30-Verbs:0
Tokens:32-Verbs:0
Tokens:62-Verbs:0
Tokens:136-Verbs:0
Tokens:55-Verbs:0
Tokens:78-Verbs:0
Tokens:23-Verbs:0
Tokens:57-Verbs:0
Tokens:57-Verbs:0
Tokens:22-Verbs:0
Tokens:17-Verbs:0
Tokens:50-Verbs:0
Tokens:23-Verbs:0
Tokens:5-Verbs:0
Tokens:73-Verbs:0
Tokens:

...

Tokens:34-Verbs:0
Tokens:5-Verbs:0
Tokens:30-Verbs:0
Tokens:11-Verbs:0
Tokens:43-Verbs:0
Tokens:91-Verbs:0
Tokens:25-Verbs:0
Tokens:57-Verbs:0
Tokens:103-Verbs:0
Tokens:20-Verbs:0
Tokens:49-Verbs:0
Tokens:58-Verbs:0
Tokens:20-Verbs:0
Tokens:43-Verbs:0
Tokens:24-Verbs:0
Tokens:87-Verbs:0
Tokens:41-Verbs:0
Tokens:5-Verbs:0
Tokens:30-Verbs:0
Tokens:40-Verbs:0
Tokens:14-Verbs:0
Tokens:5-Verbs:0
Tokens:77-Verbs:0
Tokens:63-Verbs:0
Tokens:66-Verbs:0
Tokens:8-Verbs:0
Tokens:60-Verbs:0
Tokens:20-Verbs:0
Tokens:100-Verbs:0
Tokens:46-Verbs:0
Tokens:10-Verbs:0
Tokens:60-Verbs:0
Tokens:88-Verbs:0
Tokens:94-Verbs:0
Tokens:42-Verbs:0
Tokens:5-Verbs:0
Tokens:37-Verbs:0
Tokens:77-Verbs:0
Tokens:21-Verbs:0
Tokens:16-Verbs:0
Tokens:72-Verbs:0
Tokens:58-Verbs:0
Tokens:32-Verbs:0
Tokens:21-Verbs:0
Tokens:64-Verbs:0
Tokens:22-Verbs:0
Tokens:28-Verbs:0
Tokens:65-Verbs:0
Tokens:39-Verbs:0
Tokens:25-Verbs:0
Tokens:72-Verbs:0
Tokens:35-Verbs:0
Tokens:239-Verbs:0
Tokens:47-Verbs:0
Tokens:104-Verbs:0
Tokens:14-V

Tokens:114-Verbs:0
Tokens:336-Verbs:0
Tokens:163-Verbs:0
Tokens:118-Verbs:0
Tokens:49-Verbs:0
Tokens:17-Verbs:0
Tokens:64-Verbs:0
Tokens:13-Verbs:0
Tokens:37-Verbs:0
Tokens:17-Verbs:0
Tokens:20-Verbs:0
Tokens:20-Verbs:1
Tokens:18-Verbs:0
Tokens:24-Verbs:0
Tokens:31-Verbs:0
Tokens:54-Verbs:0
Tokens:7-Verbs:0
Tokens:89-Verbs:0
Tokens:12-Verbs:0
Tokens:40-Verbs:0
Tokens:29-Verbs:0
Tokens:5-Verbs:0
Tokens:73-Verbs:0
Tokens:57-Verbs:0
Tokens:46-Verbs:0
Tokens:59-Verbs:0
Tokens:156-Verbs:0
Tokens:47-Verbs:0
Tokens:8-Verbs:0
Tokens:36-Verbs:0
Tokens:46-Verbs:0
Tokens:43-Verbs:0
Tokens:53-Verbs:0
Tokens:41-Verbs:0
Tokens:62-Verbs:0
Tokens:27-Verbs:0
Tokens:95-Verbs:0
Tokens:57-Verbs:0
Tokens:69-Verbs:0
Tokens:26-Verbs:0
Tokens:31-Verbs:0
Tokens:23-Verbs:0
Tokens:29-Verbs:0
Tokens:25-Verbs:0
Tokens:38-Verbs:0
Tokens:20-Verbs:0
Tokens:57-Verbs:0
Tokens:15-Verbs:0
Tokens:32-Verbs:0
Tokens:29-Verbs:0
Tokens:89-Verbs:0
Tokens:18-Verbs:0
Tokens:66-Verbs:0
Tokens:27-Verbs:0
Tokens:64-Verbs:0
Tokens:8

Tokens:20-Verbs:0
Tokens:49-Verbs:0
Tokens:10-Verbs:0
Tokens:94-Verbs:0
Tokens:90-Verbs:0
Tokens:94-Verbs:0
Tokens:90-Verbs:0
Tokens:98-Verbs:0
Tokens:90-Verbs:0
Tokens:96-Verbs:0
Tokens:167-Verbs:0
Tokens:79-Verbs:0
Tokens:91-Verbs:0
Tokens:972-Verbs:0
Tokens:130-Verbs:0
Tokens:51-Verbs:0
Tokens:351-Verbs:0
Tokens:11-Verbs:0
Tokens:50-Verbs:0
Tokens:83-Verbs:0
Tokens:36-Verbs:0
Tokens:21-Verbs:0
Tokens:17-Verbs:0
Tokens:26-Verbs:0
Tokens:75-Verbs:0
Tokens:119-Verbs:0
Tokens:83-Verbs:0
Tokens:83-Verbs:0
Tokens:6-Verbs:0
Tokens:16-Verbs:0
Tokens:16-Verbs:0
Tokens:22-Verbs:0
Tokens:20-Verbs:0
Tokens:20-Verbs:0
Tokens:22-Verbs:0
Tokens:24-Verbs:0
Tokens:30-Verbs:0
Tokens:18-Verbs:0
Tokens:19-Verbs:0
Tokens:43-Verbs:0
Tokens:31-Verbs:0
Tokens:18-Verbs:0
Tokens:6-Verbs:0
Tokens:54-Verbs:0
Tokens:16-Verbs:0
Tokens:16-Verbs:0
Tokens:22-Verbs:0
Tokens:20-Verbs:0
Tokens:20-Verbs:0
Tokens:22-Verbs:0
Tokens:24-Verbs:0
Tokens:30-Verbs:0
Tokens:18-Verbs:0
Tokens:19-Verbs:0
Tokens:43-Verbs:0
Tokens:

Tokens:162-Verbs:0
Tokens:150-Verbs:0
Tokens:36-Verbs:0
Tokens:134-Verbs:0
Tokens:106-Verbs:0
Tokens:33-Verbs:0
Tokens:81-Verbs:0
Tokens:26-Verbs:0
Tokens:47-Verbs:0
Tokens:100-Verbs:0
Tokens:57-Verbs:0
Tokens:106-Verbs:0
Tokens:57-Verbs:0
Tokens:150-Verbs:0
Tokens:71-Verbs:0
Tokens:47-Verbs:0
Tokens:149-Verbs:0
Tokens:164-Verbs:0
Tokens:130-Verbs:0
Tokens:102-Verbs:0
Tokens:58-Verbs:0
Tokens:119-Verbs:0
Tokens:35-Verbs:0
Tokens:94-Verbs:0
Tokens:36-Verbs:0
Tokens:88-Verbs:0
Tokens:47-Verbs:0
Tokens:25-Verbs:0
Tokens:38-Verbs:0
Tokens:124-Verbs:0
Tokens:92-Verbs:0
Tokens:33-Verbs:0
Tokens:5-Verbs:0
Tokens:27-Verbs:0
Tokens:37-Verbs:0
Tokens:97-Verbs:0
Tokens:111-Verbs:0
Tokens:146-Verbs:0
Tokens:95-Verbs:0
Tokens:17-Verbs:0
Tokens:6-Verbs:0
Tokens:26-Verbs:0
Tokens:54-Verbs:0
Tokens:82-Verbs:0
Tokens:14-Verbs:0
Tokens:25-Verbs:0
Tokens:14-Verbs:0
Tokens:32-Verbs:0
Tokens:23-Verbs:0
Tokens:84-Verbs:0
Tokens:23-Verbs:0
Tokens:53-Verbs:0
Tokens:39-Verbs:0
Tokens:35-Verbs:0
Tokens:43-Verbs

Tokens:33-Verbs:0
Tokens:17-Verbs:0
Tokens:51-Verbs:0
Tokens:25-Verbs:0
Tokens:28-Verbs:0
Tokens:37-Verbs:0
Tokens:26-Verbs:0
Tokens:48-Verbs:0
Tokens:70-Verbs:0
Tokens:70-Verbs:0
Tokens:20-Verbs:0
Tokens:11-Verbs:0
Tokens:40-Verbs:0
Tokens:37-Verbs:0
Tokens:14-Verbs:0
Tokens:40-Verbs:0
Tokens:64-Verbs:0
Tokens:33-Verbs:0
Tokens:27-Verbs:0
Tokens:11-Verbs:0
Tokens:43-Verbs:0
Tokens:58-Verbs:0
Tokens:132-Verbs:0
Tokens:50-Verbs:0
Tokens:59-Verbs:0
Tokens:16-Verbs:0
Tokens:51-Verbs:0
Tokens:8-Verbs:0
Tokens:85-Verbs:0
Tokens:27-Verbs:0
Tokens:29-Verbs:0
Tokens:44-Verbs:0
Tokens:30-Verbs:0
Tokens:27-Verbs:0
Tokens:71-Verbs:0
Tokens:28-Verbs:0
Tokens:69-Verbs:0
Tokens:61-Verbs:0
Tokens:116-Verbs:0
Tokens:8-Verbs:0
Tokens:14-Verbs:0
Tokens:63-Verbs:0
Tokens:38-Verbs:0
Tokens:46-Verbs:0
Tokens:11-Verbs:0
Tokens:29-Verbs:0
Tokens:31-Verbs:0
Tokens:59-Verbs:0
Tokens:87-Verbs:0
Tokens:11-Verbs:0
Tokens:28-Verbs:0
Tokens:37-Verbs:0
Tokens:94-Verbs:0
Tokens:14-Verbs:0
Tokens:51-Verbs:0
Tokens:74-

--------------

word2vec

lo que hacemos es utilizar una red neuronal para interrelacionar las palabras de un texto.  
Como?  

usando un autoencoder al que luego quitamos la ultima capa, esto nos da los vectores de cada palabra
una red neuronal con n entradas (palabras unicas en el texto), m neuronas en capa intermedia, y o neuronas en la salida. el sistema intenta predecir qué palabra vendrá despues de cada palabra de entrada. siendo la palabra un vector de 1Xn.  

el sistema aprenderá que palabras con vectores parecidos estan inter relacionadas (pertenecen al mismo tema)

al cortar la capa final, lo que nos queda es una matriz de datos con la probabilidad de aparicion de una palabra para las demas. eso es un vector

In [6]:
import numpy as np
import pandas as pd

w2v_model = "C:/Users/alonsoga/OneDrive - HP Inc/Oficina/Documentacion/DS/RoundU/WORD2VEC-Twitter-Espa_ol_para_Latinoam_rica__Espa_a_y_Estados_Unidos.csv"
data = pd.read_csv(w2v_model, index_col="RANK")


In [5]:
data.shape

(159747, 102)

In [8]:
from gensim.models import Word2Vec
model = Word2Vec.load(w2v_model)

UnpicklingError: unpickling stack underflow

-----------

# tutorial lda

https://nlpforhackers.io/topic-modeling/

In [83]:
from nltk.corpus import brown
 
data = []
 
for fileid in brown.fileids():
    document = ' '.join(brown.words(fileid))
    data.append(document)
 
NO_DOCUMENTS = len(data)
print(NO_DOCUMENTS)
print(data[:5])

500
["The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia's registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that Fulton legislators act `` to h

In [19]:
brown.fileids()

['ca01',
 'ca02',
 'ca03',
 'ca04',
 'ca05',
 'ca06',
 'ca07',
 'ca08',
 'ca09',
 'ca10',
 'ca11',
 'ca12',
 'ca13',
 'ca14',
 'ca15',
 'ca16',
 'ca17',
 'ca18',
 'ca19',
 'ca20',
 'ca21',
 'ca22',
 'ca23',
 'ca24',
 'ca25',
 'ca26',
 'ca27',
 'ca28',
 'ca29',
 'ca30',
 'ca31',
 'ca32',
 'ca33',
 'ca34',
 'ca35',
 'ca36',
 'ca37',
 'ca38',
 'ca39',
 'ca40',
 'ca41',
 'ca42',
 'ca43',
 'ca44',
 'cb01',
 'cb02',
 'cb03',
 'cb04',
 'cb05',
 'cb06',
 'cb07',
 'cb08',
 'cb09',
 'cb10',
 'cb11',
 'cb12',
 'cb13',
 'cb14',
 'cb15',
 'cb16',
 'cb17',
 'cb18',
 'cb19',
 'cb20',
 'cb21',
 'cb22',
 'cb23',
 'cb24',
 'cb25',
 'cb26',
 'cb27',
 'cc01',
 'cc02',
 'cc03',
 'cc04',
 'cc05',
 'cc06',
 'cc07',
 'cc08',
 'cc09',
 'cc10',
 'cc11',
 'cc12',
 'cc13',
 'cc14',
 'cc15',
 'cc16',
 'cc17',
 'cd01',
 'cd02',
 'cd03',
 'cd04',
 'cd05',
 'cd06',
 'cd07',
 'cd08',
 'cd09',
 'cd10',
 'cd11',
 'cd12',
 'cd13',
 'cd14',
 'cd15',
 'cd16',
 'cd17',
 'ce01',
 'ce02',
 'ce03',
 'ce04',
 'ce05',
 'ce06',
 

In [84]:
import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
 
NUM_TOPICS = 10
STOPWORDS = stopwords.words('english')
 
def clean_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text
 
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in data:
    tokenized_data.append(clean_text(text))
 
 
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 20th document looks like: [(word_id, count), ...]
print(corpus[20])
# [(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2),  ...
 
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 

[(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2), (44, 2), (45, 2), (46, 2), (47, 2), (49, 1), (50, 1), (53, 1), (56, 1), (59, 1), (60, 1), (66, 1), (75, 1), (80, 1), (98, 1), (101, 1), (106, 1), (117, 1), (129, 1), (130, 2), (132, 2), (135, 2), (140, 1), (141, 2), (143, 4), (144, 2), (145, 2), (166, 1), (195, 1), (198, 3), (219, 1), (220, 4), (221, 3), (223, 1), (229, 4), (230, 4), (231, 2), (235, 1), (236, 1), (242, 2), (246, 2), (255, 1), (263, 1), (269, 1), (270, 5), (271, 2), (275, 5), (276, 1), (278, 4), (280, 2), (281, 1), (307, 2), (310, 1), (311, 3), (313, 1), (314, 5), (318, 4), (322, 1), (336, 1), (338, 3), (339, 1), (340, 1), (341, 1), (345, 1), (346, 1), (351, 1), (354, 1), (355, 1), (366, 3), (368, 13), (370, 1), (372, 1), (374, 3), (377, 3), (381, 3), (386, 1), (392, 6), (396, 1), (401, 1), (412, 2), (426, 2), (428, 2), (431, 2), (434, 2), (439, 2), (444, 1), (450, 1), (452, 1), (462, 1), (465, 1), (467, 1), (470, 1), (478, 1), (483, 1), (

In [85]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)
 

LDA Model:
Topic #0: 0.005*"one" + 0.005*"would" + 0.004*"said" + 0.004*"new" + 0.003*"could" + 0.003*"man" + 0.003*"time" + 0.003*"years" + 0.002*"two" + 0.002*"like"
Topic #1: 0.005*"would" + 0.005*"one" + 0.004*"could" + 0.003*"new" + 0.003*"like" + 0.003*"said" + 0.003*"two" + 0.002*"may" + 0.002*"time" + 0.002*"first"
Topic #2: 0.006*"one" + 0.006*"would" + 0.003*"new" + 0.003*"said" + 0.003*"time" + 0.003*"could" + 0.003*"two" + 0.002*"must" + 0.002*"first" + 0.002*"man"
Topic #3: 0.006*"one" + 0.004*"would" + 0.003*"said" + 0.003*"could" + 0.003*"first" + 0.003*"much" + 0.003*"new" + 0.002*"two" + 0.002*"even" + 0.002*"also"
Topic #4: 0.006*"one" + 0.005*"would" + 0.003*"may" + 0.003*"said" + 0.003*"could" + 0.003*"man" + 0.003*"made" + 0.003*"time" + 0.002*"even" + 0.002*"two"
Topic #5: 0.006*"one" + 0.006*"would" + 0.005*"said" + 0.003*"time" + 0.003*"could" + 0.003*"new" + 0.003*"first" + 0.002*"like" + 0.002*"man" + 0.002*"two"
Topic #6: 0.006*"one" + 0.005*"would" + 0.003*"

In [86]:
text = "The economy is working better than ever"
bow = dictionary.doc2bow(clean_text(text))
 
print(lsi_model[bow])
# [(0, 0.091615426138426506), (1, -0.0085557463300508351), (2, 0.016744863677828108), (3, 0.040508186718598529), (4, 0.014201267714185898), (5, -0.012208538275305329), (6, 0.031254053085582149), (7, 0.017529584659403553), (8, 0.056957633371540077),(9, 0.025989149894888153)]
 
print(lda_model[bow])
# [(0, 0.020005183), (1, 0.020005869), (2, 0.02000626), (3, 0.020005472), (4, 0.020009108), (5, 0.020005926), (6, 0.81994385), (7, 0.020006068), (8, 0.020006327), (9, 0.020005994)]


[(0, 0.09161539802604365), (1, -0.008762916549283174), (2, -0.015621852896384462), (3, -0.041248897937389006), (4, -0.015233856142161531), (5, -0.011950203529580355), (6, -0.030807771538930977), (7, -0.019101036906422304), (8, 0.055239910037807594), (9, -0.028300180974318863)]
[(0, 0.02001635), (1, 0.020015512), (2, 0.020015247), (3, 0.0200148), (4, 0.020015126), (5, 0.020014271), (6, 0.020014463), (7, 0.020015268), (8, 0.8198643), (9, 0.020014707)]


In [87]:
from gensim import similarities
 
lda_index = similarities.MatrixSimilarity(lda_model[corpus])
 
# Let's perform some queries
similarities = lda_index[lda_model[bow]]
# Sort the similarities
similarities = sorted(enumerate(similarities), key=lambda item: -item[1])
 
# Top most similar documents:
print(similarities[:10])
# [(104, 0.87591344), (178, 0.86124849), (31, 0.8604598), (77, 0.84932965), (85, 0.84843522), (135, 0.84421808), (215, 0.84184396), (353, 0.84038532), (254, 0.83498049), (13, 0.82832891)]
 
# Let's see what's the most similar document
document_id, similarity = similarities[0]
print(data[document_id][:1000])

[(399, 0.9976162), (239, 0.9976043), (392, 0.99757266), (88, 0.9975399), (264, 0.9975301), (51, 0.99732953), (109, 0.99732953), (122, 0.99732953), (242, 0.99732953), (280, 0.99732953)]
But they all said , `` No , your time will come . Enjoy being a bride while you can '' . There was no room for company in the tiny Weaning House ( where the Albright boys always took their brides , till they could get a house and a farm of their own ) . So when the Big House filled up and ran over , the sisters-in-law found beds for everyone in their own homes . And there was still not anything that Linda Kay could do . So Linda Kay gave up asking , and accepted her reprieve . Without saying so , she was really grateful ; ; for to attend the dying was something she had never experienced , and certainly had not imagined when she thought of the duties she would have as Bobby Joe's wife . She had made curtains for all the windows of her little house , and she had kept it spotless and neat , shabby as it was

------

In [97]:
texto = []
for i in corpus:
    texto.append(corpus[i])

In [98]:
texto[0]

['@aritzeta @bibiloni @JosepMClD @Ratafia_Party @jbadia16 Jo també. Però diria que el reflexiu de ESTAR és sobrer: https://t.co/UHxFhIDU0n\nhttps://t.co/25w8rZ6vlq',
 '@CUPRiudoms Sí.\nHo he vist de casualitat. Millor un MD.',
 '@Josefina170541 @martonamg @fananwa "Quan fou mort el combregaren" equival a "A buenas horas mangas verdes".',
 '@literaturaimes @EnricGoma @martonamg "Quan fou mort el combregaren" equival a "A buenas horas mangas verdes".',
 '@mauletgroc @martonamg "Quan fou mort el combregaren" equival a "A buenas horas mangas verdes".',
 '@TrudiGonzlezMa1 @martonamg Potser sí.',
 '@victoracedoma @bibiloni @JosepMClD @Ratafia_Party @jbadia16 No, no ho és: https://t.co/NM5reDcDTi',
 '@Fakerragut Jo no.',
 '@Romeucarles @VogelfreiCAT Sí que ho és.']

In [99]:
data = []

for fileid in texto:
    document = ' '.join(fileid)
    data.append(document)
 
NO_DOCUMENTS = len(data)
print(NO_DOCUMENTS)
print(dato[:5])

3876
['@aritzeta @bibiloni @JosepMClD @Ratafia_Party @jbadia16 Jo també. Però diria que el reflexiu de ESTAR és sobrer: https://t.co/UHxFhIDU0n\nhttps://t.co/25w8rZ6vlq @CUPRiudoms Sí.\nHo he vist de casualitat. Millor un MD. @Josefina170541 @martonamg @fananwa "Quan fou mort el combregaren" equival a "A buenas horas mangas verdes". @literaturaimes @EnricGoma @martonamg "Quan fou mort el combregaren" equival a "A buenas horas mangas verdes". @mauletgroc @martonamg "Quan fou mort el combregaren" equival a "A buenas horas mangas verdes". @TrudiGonzlezMa1 @martonamg Potser sí. @victoracedoma @bibiloni @JosepMClD @Ratafia_Party @jbadia16 No, no ho és: https://t.co/NM5reDcDTi @Fakerragut Jo no. @Romeucarles @VogelfreiCAT Sí que ho és.', 'Que fuerte!!! Y que lastima Alucinante!!!! Y se queda tan pancha!!!!??????', '@Helga_tito Bro no me quejo, si me parece de puta madre es de los mejores casters en LVP.\nSolo lo comentaba. Fichadisimo https://t.co/t1esxOuPSf Los Nerfs se veían a millas. Va a

In [100]:
import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
 
NUM_TOPICS = 10
STOPWORDS = stopwords.words('spanish')

In [101]:
def clean_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text
 
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in dato:
    tokenized_data.append(clean_text(text))
 
 
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 20th document looks like: [(word_id, count), ...]
print(corpus[20])
# [(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2),  ...
 
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 

<>:3: DeprecationWarning: invalid escape sequence \-
<>:3: DeprecationWarning: invalid escape sequence \-
<>:3: DeprecationWarning: invalid escape sequence \-
<ipython-input-101-34d0d693bb9c>:3: DeprecationWarning: invalid escape sequence \-
  cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]


[(56, 1), (95, 1), (98, 1), (102, 2), (113, 1), (120, 1), (128, 1), (171, 1), (211, 1), (224, 1), (320, 1), (340, 1), (347, 1), (417, 1), (440, 1), (486, 1), (507, 3), (525, 1), (556, 1), (614, 1), (825, 2), (893, 1), (968, 1), (979, 1), (985, 1), (1123, 1), (1137, 1), (1170, 1), (1172, 1), (1230, 1), (1255, 1), (1332, 1), (1336, 1), (1343, 1), (1358, 1), (1416, 1), (1551, 1), (1597, 1), (1656, 1), (1679, 1), (1680, 1), (1775, 1), (1941, 1), (1969, 1), (1970, 1), (1971, 1), (1972, 1), (1973, 1), (1974, 1), (1975, 1), (1976, 1), (1977, 3), (1978, 1), (1979, 1), (1980, 4), (1981, 1), (1982, 1), (1983, 2), (1984, 2), (1985, 1), (1986, 1), (1987, 1), (1988, 1), (1989, 1), (1990, 1), (1991, 1), (1992, 1), (1993, 1), (1994, 1), (1995, 1), (1996, 1), (1997, 1), (1998, 1), (1999, 1), (2000, 1), (2001, 2), (2002, 1), (2003, 1), (2004, 2), (2005, 1), (2006, 1), (2007, 2), (2008, 3), (2009, 5), (2010, 3), (2011, 1), (2012, 1), (2013, 5), (2014, 1), (2015, 1), (2016, 3), (2017, 2), (2018, 1), (201

In [102]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)
 

LDA Model:
Topic #0: 0.044*"https" + 0.005*"barcelona" + 0.005*"gracias" + 0.004*"ser" + 0.004*"ahora" + 0.003*"hace" + 0.003*"siempre" + 0.003*"bien" + 0.003*"pues" + 0.003*"solo"
Topic #1: 0.019*"https" + 0.004*"mejor" + 0.004*"ver" + 0.004*"siempre" + 0.003*"ser" + 0.003*"hoy" + 0.003*"buenos" + 0.003*"bien" + 0.003*"hacer" + 0.003*"vez"
Topic #2: 0.054*"https" + 0.013*"barcelona" + 0.004*"gracias" + 0.003*"foto" + 0.003*"hoy" + 0.003*"podem_cat" + 0.003*"nivel" + 0.003*"ver" + 0.002*"spain" + 0.002*"ahora"
Topic #3: 0.023*"https" + 0.003*"barcelona" + 0.003*"solo" + 0.002*"creo" + 0.002*"hoy" + 0.002*"mejor" + 0.002*"bien" + 0.002*"gente" + 0.002*"ser" + 0.002*"vida"
Topic #4: 0.013*"https" + 0.006*"gracias" + 0.003*"pues" + 0.002*"bien" + 0.002*"ser" + 0.002*"gente" + 0.002*"hacer" + 0.002*"hoy" + 0.002*"quiero" + 0.002*"muchas"
Topic #5: 0.052*"https" + 0.009*"barcelona" + 0.005*"gracias" + 0.004*"hoy" + 0.004*"spain" + 0.003*"bien" + 0.003*"mayo" + 0.003*"mejor" + 0.003*"bcnenco

In [103]:
text = "la economia funciona mejor que nunca"
bow = dictionary.doc2bow(clean_text(text))
 
print(lsi_model[bow])
# [(0, 0.091615426138426506), (1, -0.0085557463300508351), (2, 0.016744863677828108), (3, 0.040508186718598529), (4, 0.014201267714185898), (5, -0.012208538275305329), (6, 0.031254053085582149), (7, 0.017529584659403553), (8, 0.056957633371540077),(9, 0.025989149894888153)]
 
print(lda_model[bow])
# [(0, 0.020005183), (1, 0.020005869), (2, 0.02000626), (3, 0.020005472), (4, 0.020009108), (5, 0.020005926), (6, 0.81994385), (7, 0.020006068), (8, 0.020006327), (9, 0.020005994)]
 

[(0, 0.0297811593423918), (1, -0.009411784151092489), (2, 0.14294493487571155), (3, -0.04581784470927558), (4, 0.03313260288709414), (5, -0.16441981686102528), (6, -0.011254134487083749), (7, 0.027497121329167304), (8, 0.0963556875981955), (9, -0.016229292428036623)]
[(0, 0.020040547), (1, 0.8196421), (2, 0.020037929), (3, 0.020039339), (4, 0.020040162), (5, 0.020040374), (6, 0.020039763), (7, 0.02003906), (8, 0.02003836), (9, 0.020042403)]


In [104]:
from gensim import similarities
 
lda_index = similarities.MatrixSimilarity(lda_model[corpus])
 
# Let's perform some queries
similarities = lda_index[lda_model[bow]]
# Sort the similarities
similarities = sorted(enumerate(similarities), key=lambda item: -item[1])
 
# Top most similar documents:
print(similarities[:10])
# [(104, 0.87591344), (178, 0.86124849), (31, 0.8604598), (77, 0.84932965), (85, 0.84843522), (135, 0.84421808), (215, 0.84184396), (353, 0.84038532), (254, 0.83498049), (13, 0.82832891)]
 
# Let's see what's the most similar document
document_id, similarity = similarities[0]
print(data[document_id])
 

[(1264, 1.0), (1602, 1.0), (1614, 1.0), (2136, 1.0), (2784, 1.0), (747, 0.99999994), (825, 0.99999994), (1387, 0.99999994), (2026, 0.99999976), (3487, 0.99999976)]
@Jomariga @ElMangazoTolili Jajajajaja y como fue el resultado ?


In [106]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
text = "The economy is working better than ever"
x = lda_model.transform(vectorizer.transform([text]))[0]
print(x, x.sum())

<>:8: DeprecationWarning: invalid escape sequence \-
<>:8: DeprecationWarning: invalid escape sequence \-
<>:8: DeprecationWarning: invalid escape sequence \-
<ipython-input-106-043e94897325>:8: DeprecationWarning: invalid escape sequence \-
  token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1] 1.0


In [107]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
9      -60.291897 -295.919739       1        1  49.680757
3       51.644695  347.386322       2        1  20.764395
2     -193.540131  186.349823       3        1   6.857590
8      186.045364   -8.981249       4        1   6.447281
7      399.869354  -45.102192       5        1   4.792159
4      -15.891445  -80.818169       6        1   4.762016
0      276.254181  209.617737       7        1   3.768797
1     -238.479523  -63.010460       8        1   1.561278
6      196.890076 -244.003296       9        1   0.755442
5       22.843603  129.932663      10        1   0.610286, topic_info=     Category         Freq            Term        Total  loglift  logprob
term                                                                     
1911  Default  4564.000000           https  4564.000000  30.0000  30.0000
347   Default   715.000000       barcelona   715.000000  29.0000  29.0000
2348  Default  2653.000000             los  2653.000000  28.0000  28.0000
1029  Default  1797.000000             del  1797.000000  27.0000  27.0000
3338  Default  8610.000000             que  8610.000000  26.0000  26.0000
2918  Default  1848.000000            para  1848.000000  25.0000  25.0000
1759  Default   520.000000         gracias   520.000000  24.0000  24.0000
1404  Default   543.000000            espa   543.000000  23.0000  23.0000
2214  Default  1513.000000             las  1513.000000  22.0000  22.0000
343   Default   283.000000             bar   283.000000  21.0000  21.0000
3169  Default  2333.000000             por  2333.000000  20.0000  20.0000
4155  Default  1630.000000             una  1630.000000  19.0000  19.0000
2517  Default   218.000000           messi   218.000000  18.0000  18.0000
2479  Default   420.000000           mejor   420.000000  17.0000  17.0000
3838  Default   167.000000           spain   167.000000  16.0000  16.0000
1434  Default   530.000000            esta   530.000000  15.0000  15.0000
551   Default   147.000000            casa   147.000000  14.0000  14.0000
1909  Default   404.000000             hoy   404.000000  13.0000  13.0000
1564  Default   175.000000           feliz   175.000000  12.0000  12.0000
363   Default   168.000000             bcn   168.000000  11.0000  11.0000
3816  Default   330.000000            solo   330.000000  10.0000  10.0000
4011  Default   148.000000          tienes   148.000000   9.0000   9.0000
2946  Default   210.000000         partido   210.000000   8.0000   8.0000
2639  Default   322.000000           mucho   322.000000   7.0000   7.0000
2197  Default   129.000000            krls   129.000000   6.0000   6.0000
566   Default   191.000000             cat   191.000000   5.0000   5.0000
447   Default   158.000000          buenos   158.000000   4.0000   4.0000
1592  Default   168.000000           final   168.000000   3.0000   3.0000
3741  Default   359.000000         siempre   359.000000   2.0000   2.0000
2244  Default   261.000000             les   261.000000   1.0000   1.0000
...       ...          ...             ...          ...      ...      ...
1150  Topic10     8.811295         direcci     9.757917   4.9970  -4.5800
3312  Topic10     8.620846          puedas     9.566568   4.9949  -4.6019
460   Topic10     8.323241          cabify     9.267561   4.9915  -4.6370
1467  Topic10     8.281101           etapa     9.225616   4.9910  -4.6421
3638  Topic10     8.016114      salvadostv     8.960671   4.9876  -4.6746
3533  Topic10     7.979501         retraso     8.923843   4.9871  -4.6792
1572  Topic10     7.772255             fen     8.716764   4.9843  -4.7055
1175  Topic10     7.100374           ditos     8.044710   4.9741  -4.7959
2487  Topic10     7.064710            meno     8.009206   4.9735  -4.8010
4201  Topic10     6.497459            vaso     7.441770   4.9633  -4.8847
2768  Topic10     6.543069            nova     7.495012   4.9632  -4.8777
1903 

---------

topic detection with scikitlearn

In [88]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
 
# Let's see how the first document in the corpus looks like in different topic spaces
print(lda_Z[0])
print(nmf_Z[0])
print(lsi_Z[0])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


(500, 10)
(500, 10)
(500, 10)
[1.05621406e-04 1.05611603e-04 1.05607885e-04 8.75626978e-01
 1.05615000e-04 1.05611164e-04 1.05599463e-04 1.05598046e-04
 1.23528141e-01 1.05616083e-04]
[0.         0.         2.11594243 0.07685464 0.         0.54264455
 1.06265208 0.         0.         0.24457341]
[ 23.30684283   1.5954538   21.81482173   0.1109631    0.90540291
   9.73363103   3.95281691  -1.88385942   1.55940652 -13.00565601]


In [80]:

import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

Loading BokehJS ...

In [89]:
svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(data))
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)
 

In [90]:
svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vectorized.T)
 
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorizer.get_feature_names()
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

LDA

In [92]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
text = "The economy is working better than ever"
x = lda_model.transform(vectorizer.transform([text]))[0]
print(x, x.sum())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


[0.02500003 0.02500101 0.02500614 0.02500006 0.02500378 0.02501686
 0.02500001 0.77496273 0.02500111 0.02500827] 1.0


In [93]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel
 

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
9      234.806122  177.377869       1        1  60.187870
7      279.118378  -17.269081       2        1  20.888066
4     -197.181168  -90.648140       3        1  10.779816
1      -61.513947   62.092506       4        1   3.803638
5      -23.239277 -137.163162       5        1   1.751088
8     -115.744736  259.028717       6        1   0.808179
2     -241.499222  103.991554       7        1   0.608709
3       60.868610  223.945999       8        1   0.573922
0      153.354187 -172.294754       9        1   0.400388
6       99.141670   24.671328      10        1   0.198326, topic_info=      Category         Freq        Term        Total  loglift  logprob
term                                                                  
5692   Default  1269.000000         man  1269.000000  30.0000  30.0000
6251   Default  1573.000000         new  1573.000000  29.0000  29.0000
8177   Default  1967.000000        said  1967.000000  28.0000  28.0000
6115   Default   460.000000         mrs   460.000000  27.0000  27.0000
5473   Default   663.000000        life   663.000000  26.0000  26.0000
4184   Default   303.000000         god   303.000000  25.0000  25.0000
9683   Default  1554.000000        time  1554.000000  24.0000  24.0000
2926   Default   515.000000         don   515.000000  23.0000  23.0000
5681   Default   773.000000        make   773.000000  22.0000  22.0000
10595  Default   706.000000        year   706.000000  21.0000  21.0000
4196   Default   762.000000        good   762.000000  20.0000  20.0000
4576   Default   533.000000        home   533.000000  19.0000  19.0000
9010   Default   879.000000       state   879.000000  18.0000  18.0000
5576   Default   724.000000        long   724.000000  17.0000  17.0000
6910   Default   548.000000       place   548.000000  16.0000  16.0000
10539  Default   755.000000       world   755.000000  15.0000  15.0000
10103  Default   589.000000         use   589.000000  14.0000  14.0000
5232   Default   849.000000        just   849.000000  13.0000  13.0000
5187   Default   351.000000        john   351.000000  12.0000  12.0000
2435   Default   251.000000       death   251.000000  11.0000  11.0000
2706   Default  1013.000000         did  1013.000000  10.0000  10.0000
6754   Default   812.000000      people   812.000000   9.0000   9.0000
10598  Default   926.000000       years   926.000000   8.0000   8.0000
5291   Default   668.000000        know   668.000000   7.0000   7.0000
1561   Default   326.000000      church   326.000000   6.0000   6.0000
5980   Default   242.000000        miss   242.000000   5.0000   5.0000
10104  Default   605.000000        used   605.000000   4.0000   4.0000
2906   Default   455.000000        does   455.000000   3.0000   3.0000
7180   Default   409.000000   president   409.000000   2.0000   2.0000
1605   Default   193.000000       class   193.000000   1.0000   1.0000
...        ...          ...         ...          ...      ...      ...
8945   Topic10     0.898573  sprinkling     6.670390   4.2184  -6.7659
6756   Topic10     1.362067      pepper    10.395034   4.1907  -6.3499
6885   Topic10     0.969130       pills     7.874548   4.1280  -6.6903
5829   Topic10     4.501727        meat    37.483462   4.1036  -5.1544
3852   Topic10     2.161658        foil    18.094677   4.0983  -5.8880
4364   Topic10     1.942879         ham    16.829720   4.0640  -5.9947
10563  Topic10     0.672939    wrapping     6.150463   4.0104  -7.0550
8067   Topic10     1.016736       roast     9.383267   4.0007  -6.6423
4177   Topic10     0.731455      gloves     7.101944   3.9499  -6.9716
983    Topic10     0.782734       blend     7.942848   3.9058  -6.9039
7038   Topic10     1.130565    portable    11.674776   3.8883  -6.5362
2318   Topic10     0.772781     crushed     8.867153   3.7829  -6.9167
2335   Topic10     1.107613        cups    13.012901   3.7593  -6.

-----------

## word2vec on twitter

In [20]:
import gensim

In [58]:
data.head()

,msg_id,msg_users_id,msg_timestamp,msg_text
0,1123283650638942211,428308122,1556646666570,@aritzeta @bibiloni @JosepMClD @Ratafia_Party ...
1,1123283654740983808,1669818008,1556646667548,Que fuerte!!! Y que lastima
2,1123283664123703296,2295120738,1556646669785,"@Helga_tito Bro no me quejo, si me parece de p..."
3,1123283666682220544,3327214991,1556646670395,@iescolar Evidente!!!
4,1123283688136032256,141861159,1556646675510,Karen me manda msj todos los días ??


In [65]:
data.shape

(18316, 4)

In [ ]:
corpus = {}
for i in range(data.shape[0]):
    if data.iloc[i]["msg_users_id"] in corpus:
        corpus[data.iloc[i]["msg_users_id"]].append(data.iloc[i]["msg_text"])
    else:
        corpus[data.iloc[i]["msg_users_id"]] = [data.iloc[i]["msg_text"]]
type(corpus), len(corpus)

In [38]:
for i in corpus.keys():
    corpus[i] = gensim.utils.simple_preprocess("".join(corpus[i]).encode('utf-8'))

In [21]:
corpus = {}
for i in range(data.shape[0]):
    if data.iloc[i]["msg_users_id"] in corpus:
        corpus[data.iloc[i]["msg_users_id"]].append(gensim.utils.simple_preprocess(data.iloc[i]["msg_text"].encode('utf-8')))
    else:
        corpus[data.iloc[i]["msg_users_id"]] = gensim.utils.simple_preprocess(data.iloc[i]["msg_text"].encode('utf-8'))
type(corpus), len(corpus)

(dict, 4184)

In [58]:
list(corpus.keys())[:5]

['428308122', '1669818008', '2295120738', '3327214991', '141861159']

In [60]:
for i in list(corpus.keys())[:5]:
    print(corpus[i])
    print()

['aritzeta', 'bibiloni', 'josepmcld', 'ratafia_party', 'jbadia', 'jo', 'també', 'però', 'diria', 'que', 'el', 'reflexiu', 'de', 'estar', 'és', 'sobrer', 'https', 'co', 'uhxfhidu', 'https', 'co', 'rz', 'vlq', 'cupriudoms', 'sí', 'ho', 'he', 'vist', 'de', 'casualitat', 'millor', 'un', 'md', 'josefina', 'martonamg', 'fananwa', 'quan', 'fou', 'mort', 'el', 'combregaren', 'equival', 'buenas', 'horas', 'mangas', 'verdes', 'literaturaimes', 'enricgoma', 'martonamg', 'quan', 'fou', 'mort', 'el', 'combregaren', 'equival', 'buenas', 'horas', 'mangas', 'verdes', 'mauletgroc', 'martonamg', 'quan', 'fou', 'mort', 'el', 'combregaren', 'equival', 'buenas', 'horas', 'mangas', 'verdes', 'trudigonzlezma', 'martonamg', 'potser', 'sí', 'victoracedoma', 'bibiloni', 'josepmcld', 'ratafia_party', 'jbadia', 'no', 'no', 'ho', 'és', 'https', 'co', 'nm', 'redcdti', 'fakerragut', 'jo', 'no', 'romeucarles', 'vogelfreicat', 'sí', 'que', 'ho', 'és']

['que', 'fuerte', 'que', 'se', 'queda', 'tan', 'pancha']

['helga_

In [92]:
corpus["428308122"][:5]

['aritzeta', 'bibiloni', 'josepmcld', 'ratafia_party', 'jbadia']

In [106]:
df = []
for i in corpus.keys():
    for j in corpus[i]:
        df.append(j)

In [107]:
print("List of lists. Let's confirm: ", type(df), " of ", type(df[0]))

List of lists. Let's confirm:  <class 'list'>  of  <class 'str'>


In [119]:
model = gensim.models.Word2Vec(size=150, window=10, min_count=1, sg=1, workers=10)
model.build_vocab([df])  # prepare the model vocabulary

In [109]:
#no more urls
import re
for i in range(len(df)):
    for j in range(len(df[i])):
        df[i][j] = re.sub(r'http\S+', '', df[i][j], flags=re.MULTILINE)

TypeError: 'str' object does not support item assignment

In [ ]:
model.train(sentences=df, total_examples=len(df), epochs=model.iter)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


In [ ]:
word_vectors = model.wv
count = 0
for word in word_vectors.vocab:
    if count<10:
        print(word)
        count += 1
    else:
        break

In [ ]:
len(word_vectors.vocab)
